In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/university /fall 2024/markov model/markvod_models/generated_data`


Status `~/Documents/university /fall 2024/markov model/markvod_models/generated_data/Project.toml`
  [31c24e10] Distributions v0.25.119
⌃ [e9467ef8] GLMakie v0.11.3
  [429524aa] Optim v1.12.0
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [7]:
include("dimer_difussion.jl")
include("free_difusion.jl")
include("state_generator.jl")

time_sequence_with_split (generic function with 1 method)

In [8]:

using GLMakie
function animate_particles(p1, p2, filename="particle_animation.mp4")
    n_steps = size(p1, 2)
    
    # Create figure
    fig = Figure(size=(600, 600))
    ax = Axis(fig[1, 1], aspect=DataAspect())
    
    # Calculate axis limits
    all_x = vcat(p1[1,:], p2[1,:])
    all_y = vcat(p1[2,:], p2[2,:])
    
    x_range = maximum(all_x) - minimum(all_x)
    y_range = maximum(all_y) - minimum(all_y)
    
    limits!(ax, -1, 1, -1, 1)
    
    # Create initial points
    point1 = scatter!([p1[1,1]], [p1[2,1]], color=:blue, markersize=10)
    point2 = scatter!([p2[1,1]], [p2[2,1]], color=:red, markersize=10)
    
    # Create animation
    record(fig, filename, 1:n_steps; framerate=10) do frame
        # Update particle positions for current frame
        point1[1] = [p1[1,frame]]
        point1[2] = [p1[2,frame]]
        
        point2[1] = [p2[1,frame]]
        point2[2] = [p2[2,frame]]
    end
    
    return filename
end




function reverse_columns_preserve_size(arr)
    num_rows, num_cols = size(arr)
    result = zeros(eltype(arr), num_rows, num_cols)
    
    for col in 1:num_cols
        result[:, col] = arr[:, num_cols - col + 1]
    end
    
    return result
end


reverse_columns_preserve_size (generic function with 1 method)

In [9]:
function shift(vec, shift_x, shift_y)
    return vec .+ [shift_x, shift_y]
end

shift (generic function with 1 method)

In [10]:
function path_correction(particle_path::Matrix{Float64}, new_destination::Vector{Float64})
    
    if size(particle_path, 1) != 2
        error("Particle path must be a 2×n matrix")
    end
    
    if length(new_destination) != 2
        error("New destination must be a 2-element vector")
    end
    
   
    n = size(particle_path, 2)
    
    
    start_point = particle_path[:, 1]
    original_end = particle_path[:, end]
    

    original_displacement = original_end - start_point
    desired_displacement = new_destination - start_point
    
  
    new_path = copy(particle_path)
    
    
    for i in 2:n
        
        t = (i - 1) / (n - 1)
        
       
        expected_position = start_point + original_displacement * t
        
       
        diffusion_component = particle_path[:, i] - expected_position
        
        
        new_expected_position = start_point + desired_displacement * t
        
       
        new_path[:, i] = new_expected_position + diffusion_component
    end
    
    return new_path
end

path_correction (generic function with 1 method)

# fucntion to generate the segements 

In [11]:
function segments(time_division::Vector, mode::Symbol;  D = 0.01, r = 0.2, box = 1.0, dt = 0.016)

if mode == :full && length(time_division) == 3 # all the sections 

        p1_middle, p2_middle = constrained_diffusion(steps = time_division[2],    D = D, r = r, box = box, dt = dt)

    init_post_r = ([p1_middle[1,end], p1_middle[2,end]],[p2_middle[1,end], p2_middle[2,end]])
    p1_right, p2_right = simulate_diffusion(initial_positions = init_post_r, steps = time_division[3],  D = D, box = box, dt = dt)

    init_post_l = ([p1_middle[1,1], p1_middle[2,1]], [p2_middle[1,1], p2_middle[2,1]])
    p1_left, p2_left = simulate_diffusion(initial_positions = init_post_l, steps = time_division[1], D = D, box = box, dt = dt)
    p1_left_reversed = reverse_columns_preserve_size(p1_left)
    p2_left_reversed = reverse_columns_preserve_size(p2_left)

    p1 = hcat(p1_left_reversed, p1_middle, p1_right)
    p2 = hcat(p2_left_reversed, p2_middle, p2_right)

    return p1, p2

elseif mode == :right_half && length(time_division) == 2 # middle and right half

    p1_middle, p2_middle = constrained_diffusion(steps = time_division[1],  D = D, r = r, box = box, dt = dt)

    init_post_r = ([p1_middle[1,end], p1_middle[2,end]],  [p2_middle[1,end], p2_middle[2,end]])
    p1_right, p2_right = simulate_diffusion(initial_positions = init_post_r, steps = time_division[2],  D = D, box = box, dt = dt)

    p1 = hcat(p1_middle, p1_right)
    p2 = hcat(p2_middle, p2_right)

    return p1, p2

elseif mode == :middle && length(time_division) == 1 # only middle half 
    p1_middle, p2_middle = constrained_diffusion(steps = time_division[1],  D = D, r = r, box = box, dt = dt)
    p1 = hcat(p1_middle)
    p2 = hcat(p2_middle)

    return p1, p2

elseif mode == :left_half && length(time_division) == 2 # middle and left half
    p1_middle, p2_middle = constrained_diffusion(steps = time_division[2],   D = D, r = r, box = box, dt = dt)

    init_post_l = ([p1_middle[1,1], p1_middle[2,1]],  [p2_middle[1,1], p2_middle[2,1]])
    p1_left, p2_left = simulate_diffusion(initial_positions = init_post_l,  steps = time_division[1], D = D, box = box, dt = dt)
    p1_left_reversed = reverse_columns_preserve_size(p1_left)
    p2_left_reversed = reverse_columns_preserve_size(p2_left)

    p1 = hcat(p1_left_reversed, p1_middle)
    p2 = hcat(p2_left_reversed, p2_middle)
    return p1, p2

else
    error("Invalid mode or time division")
end 
end

segments (generic function with 1 method)

In [12]:
p1,p2 = segments([10,40], :right_half, r = 0.01, box = 1.0, dt = 0.016)

([0.8235805058201994 0.8187835015937753 … 0.6965689858340873 0.727266815199028; 0.3118487938173262 0.31118454195035306 … 0.18672864507179293 0.15827529517238242], [0.814174163252794 0.8180915878261318 … 0.7734472995148892 0.7721431697739205; 0.3084545752762453 0.3211605759989189 … 0.18935213443351318 0.20173351252923374])

# function to select the modes 

In [13]:
function modes_selection(section::Vector )

    size_section = length(section)
    if size_section == 3
        return :full
    elseif size_section == 2
        if section[1].first == 1 && section[2].first == 2
            return :left_half
        elseif section[1].first == 2 && section[2].first == 1
            return :right_half
        else
            error("Invalid section combination")
        end
    elseif size_section == 1
        return :middle
    else
        error("Invalid number of sections")
    end

end 

modes_selection (generic function with 1 method)

In [14]:
k12 = 0.5
k21 = 0.15 
states, steps = simulate_states(k12, k21, 10)
transitions = get_state_transitions(states)
time_in_state = time_sequence(states)
split= time_sequence_with_split(states)



5-element Vector{Any}:
 Any[1 => 8, 2 => 364, 1 => 26]
 Any[1 => 26, 2 => 53, 1 => 12]
 Any[1 => 12, 2 => 29, 1 => 2]
 Any[1 => 3, 2 => 2, 1 => 8]
 Any[1 => 9, 2 => 107, 1 => 1]

In [15]:
modes = modes_selection(split[3])

:full

# extracrt time stamps froms segements

In [16]:
function time_stamps(time_state)

    segments = []
    for i in 1:length(time_state)
        push!(segments, time_state[i].second)
    end


    return segments
end 

time_stamps (generic function with 1 method)

In [17]:
time = time_stamps(split[1])

3-element Vector{Any}:
   8
 364
  26

# function to modify array for path correction 

In [18]:
function path_correction!(particle_path, new_destination::Vector{Float64})
    if size(particle_path, 1) != 2
        error("Particle path must be a 2×n matrix")
    end
    
    if length(new_destination) != 2
        error("New destination must be a 2-element vector")
    end
    
    n = size(particle_path, 2)
    
    start_point = particle_path[:, 1]
    original_end = particle_path[:, end]
    
    original_displacement = original_end - start_point
    desired_displacement = new_destination - start_point
    
    for i in 2:n
        t = (i - 1) / (n - 1)
        
        expected_position = start_point + original_displacement * t
        
        diffusion_component = particle_path[:, i] - expected_position
        
        new_expected_position = start_point + desired_displacement * t
        
        particle_path[:, i] = new_expected_position + diffusion_component
    end
    
    return nothing
end

path_correction! (generic function with 1 method)

# function to genrate the whole simulation

In [19]:
function simulation(k_in ,k12_off,changes)

    k12 = k_in
    k21 = k12_off
    states, steps = simulate_states(k12, k21, changes)
    time_in_state = time_sequence_with_split(states)
    split = time_sequence(states)
    
    
    current_time  = 0 
    Particle_1 = []
    Particle_2 = []

    for i in 1:length(time_in_state)

        section_time_stamps = time_stamps(time_in_state[i])
        modes = modes_selection(section_time_stamps)
        println("section_time_stamps: ", section_time_stamps)
        println("modes: ", modes)
        if i == 1
            p1,p2= segments(section_time_stamps, modes, r = 0.01, box = 1.0, dt = 0.016)
            Particle_1 = p1
            Particle_2 = p2
            # println("p1: ", size(p1))
            # println("p2: ", size(p2))
        else 
            p1_temp, p2_temp = segments(section_time_stamps, modes, r = 0.01, box = 1.0, dt = 0.016)
            # println("p1_temp: ", size(p1_temp))
            # println("p2_temp: ", size(p2_temp))
            shift_x = -p1_temp[1,1] + Particle_1[1,end]
            shift_y = -p1_temp[2,1] + Particle_1[2,end]
            # println("shift_x: ", shift_x)
            # println("shift_y: ", shift_y)
            p1= shift(p1_temp, shift_x, shift_y)
            p2 = shift(p2_temp, shift_x, shift_y)
            # println("p1: ", size(p1[:,1]))
            # println("p2: ", size(p2[:,1]))
            # println("time_in_state: ", time_in_state[i])
            segment_view =  @view p2[:,1:time_in_state[i][1].second]
        
            path_correction!(segment_view, [Particle_2[1,end], Particle_2[2,end]])
            # println("part_1: ", size(Particle_1))
            # println("part_2: ", size(Particle_2))
            Particle_1 = hcat(Particle_1, p1)
            Particle_2 = hcat(Particle_2, p2)

        end


        
        curren_time = length(Particle_1) 

    end
   
    
    return Particle_1, Particle_2


end 

simulation (generic function with 1 method)

In [20]:
p1,p2 = simulation(0.5, 0.5, 6)

section_time_stamps: Any[43, 57, 26]
modes: full
section_time_stamps: Any[27, 22, 16]
modes: full
section_time_stamps: Any[16, 20, 1]
modes: full


([0.8263704358191007 0.8413955639601554 … 0.3637836245874976 0.3637836245874976; 0.2943330527264942 0.2996334725208815 … 0.06348325373669639 0.06348325373669639], [0.652170143024373 0.6530030589093244 … 0.3707091944419717 0.3707091944419717; 0.014445798374504461 0.0125950557245295 … 0.056269622044490375 0.056269622044490375])

In [21]:

animate_particles(p1, p2, "total_animation_segments_1_2.mp4")



"total_animation_segments_1_2.mp4"

# implementation on the forward algorithm

In [22]:
function compute_free_density(particle1::Matrix{Float64}, particle2::Matrix{Float64}, index::Int; sigma=0.1, dt=0.01)


# Calculate relative positions for current frame
Δxn = particle1[1, index] - particle2[1, index]  # x1 - x2 at current time
Δyn = particle1[2, index] - particle2[2, index]  # y1 - y2 at current time

# Calculate relative positions for previous frame
Δxn_1 = particle1[1, index-1] - particle2[1, index-1]  # x1 - x2 at previous time
Δyn_1 = particle1[2, index-1] - particle2[2, index-1]  # y1 - y2 at previous time

# Calculate squared distances and distances
dn_1square = (Δxn_1^2) + (Δyn_1^2)
dn_1 = sqrt(dn_1square)
dn_square = (Δxn^2) + (Δyn^2)
dn = sqrt(dn_square)

# Calculate density using modified Bessel function
density_val = (dn/sigma^2) * (exp((-(dn_square) - (dn_1square))/sigma^2)) * modified_bessel(dt, dn, dn_1, sigma)

# Handle edge cases with numerical integration
if density_val == Inf || isnan(density_val) || density_val == 0
density_val = 0
for θ in 0:dt:2*pi
density_val += exp(-((dn_square) + (dn_1square) - (2*dn*dn_1*cos(θ)))/(2*sigma^2))
end 
density_val *= (dn/sigma^2)*dt
end

return density_val
end

compute_free_density (generic function with 1 method)

In [23]:
function forward_algorithm(particle1::Matrix{Float64}, particle2::Matrix{Float64}, d_dimer::Float64, param::Vector{Float64}; dt=0.01, sigma=0.1)

    N = min(size(particle1, 2), size(particle2, 2)) - 1
 
    alpha = zeros(2, N)
    scale = zeros(N)
    
    # Ensure we don't go out of bounds when calculating initial state probabilities
    frame_index = min(2, N)
    

    e1_initial = compute_free_density(particle1, particle2, frame_index, sigma=sigma, dt=dt)
    
    e2_initial = compute_dimer_density(particle1, particle2, frame_index, d_dimer, sigma=sigma, dt=dt)
    
    alpha[1, 1] = e1_initial
    alpha[2, 1] = e2_initial
    
    scale[1] = sum(alpha[:, 1])
    alpha[:, 1] ./= max(scale[1], eps()) # Avoid division by zero
    
   
    k_on = param[1]
    k_off = param[2]
    T = [1-(k_on*dt) k_on*dt; 
         k_off*dt 1-(k_off*dt)]
    
    
    for t in 2:N
       
        frame_index = min(t + 2, size(particle1, 2))
        
        
        e1 = compute_free_density(particle1, particle2, frame_index, sigma=sigma, dt=dt)
        e2 = compute_dimer_density(particle1, particle2, frame_index, d_dimer, sigma=sigma, dt=dt)
        
        
        alpha[1, t] = (alpha[1, t-1]*T[1,1] + alpha[2, t-1]*T[2,1]) * e1
        alpha[2, t] = (alpha[1, t-1]*T[1,2] + alpha[2, t-1]*T[2,2]) * e2
        
        
        scale[t] = sum(alpha[:, t])
        alpha[:, t] ./= max(scale[t], eps()) 
    end
    
    
    loglikelihood = sum(log.(max.(scale, eps()))) 
    
    return alpha, loglikelihood
end

forward_algorithm (generic function with 1 method)

In [24]:
function compute_dimer_density(particle1::Matrix{Float64}, particle2::Matrix{Float64},index::Int, d_dimer::Float64; sigma=0.1, dt=0.01)

Δxn = particle1[1, index] - particle2[1, index]  
Δyn = particle1[2, index] - particle2[2, index]  


dn_square = (Δxn^2) + (Δyn^2)
dn = sqrt(dn_square)


density_val = (dn/sigma^2) * exp((-(d_dimer^2) - (dn_square)/sigma^2)) * modified_bessel(dt, d_dimer, dn, sigma)


if density_val == Inf || isnan(density_val) || density_val == 0
density_val = 0
for θ in 0:dt:2*pi
density_val += exp(-((dn_square) + (d_dimer^2) - (2*dn*d_dimer*cos(θ)))/(2*sigma^2))
end 
density_val *= (dn/sigma^2)*dt
end

return density_val
end

compute_dimer_density (generic function with 1 method)

In [25]:
function modified_bessel(dt, d1, d2, σ)
    result_me = 0.0

    x = (d1*d2)/(σ^2)

    for θ in 0:dt:2*pi
        result_me += exp(x * cos(θ))
    end

    result_me  *= dt / (2*pi)

    return result_me 
end

modified_bessel (generic function with 1 method)

In [31]:
alpha, loglikelihood = forward_algorithm(p1, p2, 0.01, [0.5, 0.5])

([0.1555698408431952 0.03083966257581771 … 0.08489794505354636 0.08824914558692568; 0.8444301591568049 0.9691603374241823 … 0.9151020549464537 0.9117508544130744], -46.720402110571236)

In [39]:
using Optim

function objective_function(param, p1, p2, d_dimer)
    _, loglikelihood = forward_algorithm(p1, p2, d_dimer, param)
    return -loglikelihood
end


initial_params = [0.1, 0.1]


function wrapper(param)
    return objective_function(param, p1, p2, 0.01)
end

result = optimize(wrapper, initial_params, NelderMead())


optimal_params = Optim.minimizer(result)
k_on_optimized = optimal_params[1]
k_off_optimized = optimal_params[2]


_, final_loglikelihood = forward_algorithm(p1, p2, 0.01, optimal_params)


([0.1555698408431952 -0.20234698909555582 … -100.51092600875394 -100.51092600875307; 0.8444301591568049 1.2023469890955558 … 101.51092600875394 101.51092600875307], 116378.1503451103)

In [40]:
k_on_optimized

2.3220834444533128e225

In [41]:
k_off_optimized

-7.069503946570523e225